LGBMで、欠損値を埋めて学習する
BillBins消す

In [42]:
import pandas as pd
import numpy as np

In [65]:
train = pd.read_csv('../../dataset/train.csv')
test = pd.read_csv('../../dataset/test.csv')
# 前処理を一度にやるためにtrainとtestをconcatする
test['Transported'] = np.nan
train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)

# split on `/` to cols (deck/num/side)
def split_cabin(df):
    cabin = df['Cabin'].str.split('/', expand=True).rename(columns={0: 'CabinDeck', 1: 'CabinNum', 2: 'CabinSide'})
    cabin['CabinNum'] = cabin['CabinNum'].astype(float)
    return pd.concat([df, cabin], axis=1)

# group passenger or not
def make_group(df):
    df['GroupId'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['PeopleId'] = df['PassengerId'].apply(lambda x: x.split('_')[1])
    df['IsGroup'] = df['GroupId'].duplicated(keep=False)
    return df

# total room service, etc...
def total_bill(df):
    df['TotalBill'] = df[
        ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    return df

# binalize 3 classes
def binalize_bill(df, th1=5000, th2=20000): 
    df = total_bill(df)
    df['BillBins'] = df['TotalBill'].apply(
        lambda x: 0 if x < th1 else (2 if x > th1 and x < th2 else 3))
    return df

def is_adult(df):
    df['IsAdult'] = df['Age'] >= 18
    return df

train_test = split_cabin(train_test)
train_test = make_group(train_test)
train_test = total_bill(train_test)
# train_test = binalize_bill(train_test)
train_test = is_adult(train_test)

train_test.head()

/var/folders/zp/6qwnpvfn0cs2whczwk_5pvqh0000gs/T/ipykernel_3936/4064627453.py:5: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Transported,CabinDeck,CabinNum,CabinSide,GroupId,PeopleId,IsGroup,TotalBill,IsAdult
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,Maham Ofracculy,0.0,B,0.0,P,0001,01,False,0.0,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,Juanna Vines,1.0,F,0.0,S,0002,01,False,736.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,Altark Susent,0.0,A,0.0,S,0003,01,True,10383.0,True
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,Solam Susent,0.0,A,0.0,S,0003,02,True,5176.0,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,Willy Santantines,1.0,F,1.0,S,0004,01,False,1091.0,False


### 使う特徴量を選ぶ

In [66]:
train_test = train_test[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'CabinDeck', 'CabinNum', 'CabinSide', 'IsGroup', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalBill', 'IsAdult', 'Transported']]
train_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinNum,CabinSide,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,IsAdult,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,B,0.0,P,False,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,F,0.0,S,False,109.0,9.0,25.0,549.0,44.0,736.0,True,1.0
2,Europa,False,TRAPPIST-1e,58.0,True,A,0.0,S,True,43.0,3576.0,0.0,6715.0,49.0,10383.0,True,0.0
3,Europa,False,TRAPPIST-1e,33.0,False,A,0.0,S,True,0.0,1283.0,371.0,3329.0,193.0,5176.0,True,0.0
4,Earth,False,TRAPPIST-1e,16.0,False,F,1.0,S,False,303.0,70.0,151.0,565.0,2.0,1091.0,False,1.0


### 欠損値を埋める

- HomePlanet→最頻値
- CryoSleep→最頻値
- Destination→最頻値
- Age→中央値で埋める
- VIP→VIPなしで埋める
- CabinNum→最頻値で埋める
- CabinSide→CabinNum==82となっているCabinSideの最頻値で埋める？→Pで埋める

In [67]:
for col in train_test.columns:
    if col == 'Transported':
        continue
    elif col == 'CabinSide':
        train_test[col] = train_test[col].fillna('P')
    elif col == 'VIP':
        train_test[col] = train_test[col].fillna(False)
    elif col == 'Age':
        train_test[col] = train_test[col].fillna(train_test[col].median())
    else:
        train_test[col] = train_test[col].fillna(train_test[col].mode()[0])

In [68]:
train_test.isna().any()

HomePlanet      False
CryoSleep       False
Destination     False
Age             False
VIP             False
CabinDeck       False
CabinNum        False
CabinSide       False
IsGroup         False
RoomService     False
FoodCourt       False
ShoppingMall    False
Spa             False
VRDeck          False
TotalBill       False
IsAdult         False
Transported      True
dtype: bool

In [69]:
train_test

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinNum,CabinSide,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,IsAdult,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,B,0.0,P,False,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,F,0.0,S,False,109.0,9.0,25.0,549.0,44.0,736.0,True,1.0
2,Europa,False,TRAPPIST-1e,58.0,True,A,0.0,S,True,43.0,3576.0,0.0,6715.0,49.0,10383.0,True,0.0
3,Europa,False,TRAPPIST-1e,33.0,False,A,0.0,S,True,0.0,1283.0,371.0,3329.0,193.0,5176.0,True,0.0
4,Earth,False,TRAPPIST-1e,16.0,False,F,1.0,S,False,303.0,70.0,151.0,565.0,2.0,1091.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,TRAPPIST-1e,34.0,False,G,1496.0,S,True,0.0,0.0,0.0,0.0,0.0,0.0,True,NaN
12966,Earth,False,TRAPPIST-1e,42.0,False,F,82.0,P,False,0.0,847.0,17.0,10.0,144.0,1018.0,True,NaN
12967,Mars,True,55 Cancri e,27.0,False,D,296.0,P,False,0.0,0.0,0.0,0.0,0.0,0.0,False,NaN
12968,Europa,False,TRAPPIST-1e,27.0,False,D,297.0,P,False,0.0,2680.0,0.0,0.0,523.0,3203.0,False,NaN


### Encoding

In [70]:
# HomePlanet, Destination, CabinSideはonehot encoding
for col in ['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide']:
    train_test = pd.concat([train_test,  pd.get_dummies(train_test[col], prefix=col)], axis=1)
    train_test = train_test.drop(col, axis=1)

In [71]:
# boolをintへ
def bool2int(df):
    for col in df.columns:
        if df[col].dtype == bool:
            df[col] = df[col].astype(float)
        if df[col].dtype == 'object':
            df[col] = df[col].map({True: 1, False: 0})
    return df

train_test = bool2int(train_test)

In [72]:
train_test.columns

Index(['CryoSleep', 'Age', 'VIP', 'CabinNum', 'IsGroup', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalBill', 'IsAdult',
       'Transported', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'CabinDeck_A',
       'CabinDeck_B', 'CabinDeck_C', 'CabinDeck_D', 'CabinDeck_E',
       'CabinDeck_F', 'CabinDeck_G', 'CabinDeck_T', 'CabinSide_P',
       'CabinSide_S'],
      dtype='object')

### モデリング

In [73]:
import lightgbm as lgbm

In [74]:
params = {
    'objective': 'binary',
    'num_leaves': 64,
    'min_child_samples': 20,
    'max_depth': 7,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'importance_type': 'gain',
}

### 学習

In [75]:
import os
import random
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score

In [76]:
def set_seed(seed=3407):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


SEED = 3407
set_seed(SEED)

In [77]:
# DataFrameをnp.ndarrayに変換
trainval = train_test[~train_test['Transported'].isna()]
test = train_test[train_test['Transported'].isna()]
# inputとlabelに分離
x_trainval = trainval.drop('Transported', axis=1).values
y_trainval = trainval.Transported.values
x_test = test.drop('Transported', axis=1).values

In [78]:
x_trainval.shape, y_trainval.shape, x_test.shape

((8693, 28), (8693,), (4277, 28))

In [79]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

val_scores = []
models = []
for fold, (train_inds, val_inds) in enumerate(kf.split(x_trainval)):
    
    x_train, x_val = x_trainval[train_inds], x_trainval[val_inds]
    y_train, y_val = y_trainval[train_inds], y_trainval[val_inds]
    
    model = lgbm.LGBMClassifier(**params)
    model.fit(
        x_train, y_train,
        eval_set=(x_val, y_val),
        eval_metric='logloss',
        callbacks=[lgbm.early_stopping(10)],
    )

    y_val_pred = model.predict(x_val)
    score = accuracy_score(y_val, y_val_pred)
    print(f'fold {fold}/acc: {score}')
    val_scores.append(score)
    models.append(model)

cv_score = np.mean(val_scores)
print(f'CV score: {cv_score}')

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[62]	valid_0's binary_logloss: 0.39207
fold 0/acc: 0.8102357676825762
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[87]	valid_0's binary_logloss: 0.384456
fold 1/acc: 0.80448533640023
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[85]	valid_0's binary_logloss: 0.385292
fold 2/acc: 0.8125359401955147
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[72]	valid_0's binary_logloss: 0.371262
fold 3/acc: 0.810126582278481
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[56]	valid_0's binary_logloss: 0.396812
fold 4/acc: 0.807825086306099
CV score: 0.8090417425725802


In [80]:
importance = pd.DataFrame()

for i in range(len(models)):
    df = pd.DataFrame(models[i].feature_importances_,
                      index=trainval.columns[:-1], columns=[f'model{i+1}'])
    df = df.sort_values(f'model{i+1}', ascending=False)
    importance = pd.concat([importance, df], axis=1)

importance

,model1,model2,model3,model4,model5
TotalBill,9724.872442,10718.486645,10097.016018,10221.485364,10472.686751
FoodCourt,2693.789607,2664.843794,2619.352106,2628.425048,2606.375670
CabinNum,2410.339130,2978.381183,3098.509894,2971.593844,2366.620543
Transported,2098.322588,2218.103082,2163.213788,2131.230835,1995.353256
Spa,1967.046513,2202.998879,2137.856788,2035.679899,1938.862660
VRDeck,1919.884985,1939.321694,2016.758104,1977.122852,1824.066406
ShoppingMall,1861.308490,1833.231737,2087.715973,2062.175874,1698.787937
Age,1222.433758,1448.917921,1550.938770,1299.935344,1190.320416
CabinDeck_D,1093.813573,1044.811061,1017.600267,1025.663458,1003.219825
RoomService,1081.814444,1211.129675,1194.204164,1033.816467,1031.127386


### submit用のcsv作成

cvごとの推論の単純平均

In [81]:
y_preds = []

for i in range(len(models)):
    predictor = models[i]
    y_pred = predictor.predict_proba(x_test)
    y_preds.append(y_pred)

ensemble = np.argmax(np.mean(y_preds, axis=0), axis=-1)

In [82]:
test_ids = pd.read_csv('../../dataset/test.csv')['PassengerId']

df_submit = pd.DataFrame(ensemble, index=test_ids, columns=['Transported'])
df_submit.Transported = df_submit.Transported.astype(bool)

In [83]:
df_submit

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [62]:
df_submit.to_csv('submission/lgbm_trial9.csv')